# Multi-Object Tracking using YOLOv8n

## Objective
To track multiple vehicles across video frames using YOLOv8n and analyze identity consistency over time.


In [22]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import defaultdict

In [23]:
model=YOLO("yolov8n.pt")

In [24]:
video_path=r"C:\Users\vpokh\Downloads\Highway_Accident_Video_Generation.mp4"
cap=cv2.VideoCapture(video_path)
assert cap.isOpened()
track_history=defaultdict(list)

In [25]:
output_path = "outputs/annotated_video.mp4"

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    output_path,
    fourcc,
    fps,
    (width, height)
)


In [27]:
while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        break
    results=model.track(frame,
                       persist=True,
                       conf=0.4,
                       classes=[2,3,5,7],
                        verbose=False
                       )
    annotated_frame=results[0].plot()
    out.write(annotated_frame)


    if results[0].boxes.id is not None:
        boxes=results[0].boxes.xyxy.cpu().numpy()
        track_ids=results[0].boxes.id.cpu().numpy()

        for box,track_id in zip(boxes,track_ids):
            x1,y1,x2,y2=box
            cx=int((x1+x2)/2)
            cy=int((y1+y2)/2)
            track_history[int(track_id)].append((cx,cy))
            
            cv2.circle(
                annotated_frame,
                (cx,cy),
                3,
                (0,255,0),
                -1
            )

            cv2.putText(
                annotated_frame,
                f"ID{int(track_id)}",
                (cx,cy-7),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                1
            )
    cv2.imshow("Vehicle Tracking",annotated_frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
out.release()        
cap.release()
cv2.destroyAllWindows()

            
    

## Observations

- Vehicle IDs remain consistent across consecutive frames.
- ID switches may occur during occlusion or close vehicle interactions.
- Tracking enables temporal analysis required for trajectory and behavior modeling.
